<a href="https://colab.research.google.com/github/TayJen/H_Tatarsan2022/blob/main/H_Tatarstan2022_Yolov7x_Multi_data_preparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данном ноутбуке помимо использования доступных на соревновании данных будет использован общедоступный датасет с сайта: [CrowdHuman](https://www.crowdhuman.org/)

При обучении модели на столь обильных данных используется свой общий диск, с огромной памятью, вся информация будет сохранена на нем

### Подготовка диска + скачивание файлов

In [1]:
%cd /content/drive/Shareddrives/data_drive
!pwd

/content/drive/Shareddrives/data_drive
/content/drive/Shareddrives/data_drive


In [2]:
!mkdir H_Tatarstan2022
%cd ./H_Tatarstan2022
!pwd

/content/drive/Shareddrives/data_drive/H_Tatarstan2022
/content/drive/Shareddrives/data_drive/H_Tatarstan2022


Открывая данные на сайте можно проверить уникальные айди файлов, для того чтобы скачать их здесь, без ручного скачивания.

Айди файлов:


*   Train Part #1: `134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y`
*   Train Part #2: `17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla`
*   Train Part #3: `1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW`
*   Annotation Train: `1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3`



Вспомогательные ссылки:


*   [Downloading data from a shared google drive link in google colab](https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab)
*   [Gdown official documentation](https://github.com/wkentaro/gdown)

In [5]:
import gdown

ids = ['134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y',
       '17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla',
       '1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW',
       '1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3']

for id in ids:
    gdown.download(id=id, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/CrowdHuman_train01.zip
100%|██████████| 2.97G/2.97G [00:37<00:00, 79.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/CrowdHuman_train02.zip
100%|██████████| 3.09G/3.09G [00:57<00:00, 53.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/CrowdHuman_train03.zip
100%|██████████| 2.31G/2.31G [00:34<00:00, 67.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/annotation_train.odgt
100%|██████████| 80.0M/80.0M [00:02<00:00, 31.8MB/s]


In [6]:
!ls

annotation_train.odgt	CrowdHuman_train02.zip
CrowdHuman_train01.zip	CrowdHuman_train03.zip


In [7]:
!mkdir ./add_data
!mkdir ./add_data/zipped
!mkdir ./add_data/unzipped
!mkdir ./add_data/annotations
!mv CrowdHuman_train01.zip ./add_data/zipped
!mv CrowdHuman_train02.zip ./add_data/zipped
!mv CrowdHuman_train03.zip ./add_data/zipped
!mv annotation_train.odgt ./add_data/annotations

In [ ]:
!unzip ./add_data/zipped/CrowdHuman_train01.zip -d ./add_data/unzipped
!unzip ./add_data/zipped/CrowdHuman_train02.zip -d ./add_data/unzipped
!unzip ./add_data/zipped/CrowdHuman_train03.zip -d ./add_data/unzipped

## Конвертация аннотаций в Yolo формат

Далее нам необходимо трансформировать используемые аннотации в Yolo формат, так как обучаться будет state-of-the-art Yolov7x

Вспомогательные ссылочки:


*   [Convert CrowdHuman dataset to Yolo annotations](https://gist.github.com/adujardin/62653118466962264aa0c6339c3e9cf5)
*   [crowdhuman_train_anno.py](https://github.com/alaksana96/darknet-crowdhuman/blob/master/crowdhuman_train_anno.py)

In [9]:
!mkdir ./add_data/unzipped/Yolo_labels

In [11]:
import numpy
import cv2
from multiprocessing import Process, Manager
import yaml
from tqdm import tqdm

intPERSON = 0
intHEAD   = 1

manager = Manager()
return_dict = manager.dict()
return_dict

<DictProxy object, typeid 'dict' at 0x7f7d08f19210>

In [12]:
def generate_annotations(ii, return_dict):
    line = return_dict[ii]
    del return_dict[ii]
    
    dictLine = yaml.load(line)

    strID = dictLine['ID']

    img = cv2.imread('./add_data/unzipped/Images/{}.jpg'.format(strID), 1)

    imgWidth  = img.shape[1]
    imgHeight = img.shape[0]

    # Create .txt label file
    with open('./add_data/unzipped/Yolo_labels/{}.txt'.format(strID), 'w+') as txtf:

        for label in dictLine['gtboxes']:

            if 'extra' in label and 'ignore' in label['extra'] and label['extra']['ignore'] == 1:
                continue
            if 'extra' in label and 'unsure' in label['extra'] and label['extra']['unsure'] == 1:
                continue 

            # Person BB
            px = float(label['fbox'][0])
            py = float(label['fbox'][1])
            pw = float(label['fbox'][2])
            ph = float(label['fbox'][3])

            # Head BB
            hx = float(label['hbox'][0])
            hy = float(label['hbox'][1])
            hw = float(label['hbox'][2])
            hh = float(label['hbox'][3])

            # Absolute person BB
            cpx = px + pw / 2
            cpy = py + ph / 2

            abspx = cpx / imgWidth
            abspy = cpy / imgHeight
            abspw = pw / imgWidth
            absph = ph / imgHeight  

            abspx = 1 if abspx > 1 else abspx
            abspy = 1 if abspy > 1 else abspy
            abspw = 1 if abspw > 1 else abspw
            absph = 1 if absph > 1 else absph
            abspx = 0.000001 if abspx < 0 else abspx
            abspy = 0.000001 if abspy < 0 else abspy
            abspw = 0.000001 if abspw < 0 else abspw
            absph = 0.000001 if absph < 0 else absph

            # Absolute head BB
            chx = hx + hw / 2
            chy = hy + hh / 2

            abshx = chx / imgWidth
            abshy = chy / imgHeight
            abshw = hw / imgWidth
            abshh = hh / imgHeight  

            abshx = 1 if abshx > 1 else abshx
            abshy = 1 if abshy > 1 else abshy
            abshw = 1 if abshw > 1 else abshw
            abshh = 1 if abshh > 1 else abshh
            abshx = 0.000001 if abshx < 0 else abshx
            abshy = 0.000001 if abshy < 0 else abshy
            abshw = 0.000001 if abshw < 0 else abshw
            abshh = 0.000001 if abshh < 0 else abshh

            # Write to file
            txtf.write('{} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(intPERSON,
                                                                 abspx,
                                                                 abspy,
                                                                 abspw,
                                                                 absph))

            txtf.write('{} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(intHEAD,
                                                                 abshx,
                                                                 abshy,
                                                                 abshw,
                                                                 abshh))

In [ ]:
with open('./add_data/annotations/annotation_train.odgt') as f:
        
    processes = []
    max_iter = 500

    for ii, line in tqdm(enumerate(f)): 
        return_dict[ii] = line
        pcs = Process(target=generate_annotations,
                      args=(ii, return_dict))
        processes.append(pcs)
        pcs.start()
        
        if ii % max_iter == 0:
            for jj in range(len(processes)):    
                processes[jj].join()  

            processes = []
            
    for jj in range(len(processes)):    
        processes[jj].join()      
        
    processes = []

10825it [24:28, 10.29it/s]